In [101]:
import pyodbc
import os
import pandas as pd
from zipfile import ZipFile

import numpy as np
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
from calendar import month_name, different_locale

# mdb_to_df

In [102]:
def mdb_to_df(file_name, sql):

    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        fr'DBQ={file_name};'
    )
    print(conn_str)
    cnxn = pyodbc.connect(conn_str)

    df = pd.read_sql(sql, cnxn)

    print(f'{file_name} Loaded')
    return df

# Read SUM

In [103]:
def read_sum(period):
    usecols_sum = """
    SELECT CDbl(TimeOn) AS TOn, CDbl(TimeOff) AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NOT NULL
    union
    SELECT CDbl(TimeOn) AS TOn, TimeOff AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NULL
    """
    
    file_name = f'../DATA/SUM/{period}-sum.mdb'

    alarms = mdb_to_df(file_name=file_name, sql=usecols_sum)

    alarms['TOn'] = sqldate_to_datetime(alarms['TOn'])
    alarms['TOff'] = sqldate_to_datetime(alarms['TOff'])

    alarms.rename(columns={'TOn': 'TimeOn',
                           'TOff': 'TimeOff'}, inplace=True)

    alarms = alarms[alarms.StationNr >= 2307405]

    alarms = alarms[
        alarms.StationNr <= 2307535].reset_index(
        drop=True)

    alarms.dropna(subset=['Alarmcode'], inplace=True)

    alarms.reset_index(drop=True, inplace=True)

    alarms.Alarmcode = alarms.Alarmcode.astype(int)

    return alarms


def sqldate_to_datetime(column):
    
    try:

        column = column.str.replace(',', '.').astype(float)
    except:
        pass

    day_parts = np.modf(column.loc[~column.isna()])

    column = column.fillna(pd.NaT)
    
    column.loc[column.notna()] = (
        dt(1899, 12, 30) +
        day_parts[1].astype('timedelta64[D]', errors='ignore') +
        (day_parts[0] * 86400000).astype('timedelta64[ms]', errors='ignore')
    )

    column = column.fillna(pd.NaT)
    
    
    return column

# Cascade

In [104]:
# Determine alarms real periods
def cascade(df):

    df.reset_index(inplace=True, drop=True)
    df['TimeOffMax'] = df.TimeOff.cummax().shift()

    df.at[0, 'TimeOffMax'] = df.at[0, 'TimeOn']

    return df


# looping through turbines and applying cascade method
def apply_cascade(result_sum):

    # Sort by alarm ID
    result_sum.sort_values(['ID'], inplace=True)
    df = result_sum.groupby('StationId').apply(cascade)

    mask_root = (df.TimeOn.values >= df.TimeOffMax.values)
    mask_children = (df.TimeOn.values < df.TimeOffMax.values) & (
        df.TimeOff.values > df.TimeOffMax.values)
    mask_embedded = (df.TimeOff.values <= df.TimeOffMax.values)

    df.loc[mask_root, 'NewTimeOn'] = df.loc[mask_root, 'TimeOn']
    df.loc[mask_children, 'NewTimeOn'] = df.loc[mask_children, 'TimeOffMax']
    df.loc[mask_embedded, 'NewTimeOn'] = df.loc[mask_embedded, 'TimeOff']

    df.drop(columns=['TimeOffMax'], inplace=True)

    df.reset_index(inplace=True, drop=True)

    TimeOff = df.TimeOff
    NewTimeOn = df.NewTimeOn

    df['RealPeriod'] = abs(TimeOff - NewTimeOn)

    mask_siemens = (df['Error Type'] == 1)
    mask_tarec = (df['Error Type'] == 0)

    df['Period Siemens(s)'] = df[mask_siemens].RealPeriod  # .dt.seconds
    df['Period Tarec(s)'] = df[mask_tarec].RealPeriod  # .dt.seconds
    # df['RealPeriod'] = df['RealPeriod'].dt.seconds

    return df

# Periods

In [2]:
def get_month_name(month_no, locale):
    with different_locale(locale):
        return month_name[month_no].capitalize()
    

period = input('period ex: "2020-07"') or '2020-07'

period_dt = dt.strptime(period, "%Y-%m")
period_month = period_dt.month
period_month_name = get_month_name(period_month, 'french')
period_year = period_dt.year

days_in_period = pd.Period(f'{period}').days_in_month

previous_period_dt = period_dt + relativedelta(months=-1)
previous_period = previous_period_dt.strftime("%Y-%m")

lastday_period_dt = period_dt + relativedelta(months=1, days=-1)
lastday_period = lastday_period_dt.strftime("%Y-%m-%d")


lastday_previous_period_dt = period_dt + relativedelta(days=-1)
lastday_previous_period = lastday_previous_period_dt.strftime("%Y-%m-%d")

days_to_previous_period = len(pd.date_range(
    f'{period_year}-01-01', lastday_previous_period_dt))

In [387]:
days_in_period


30

# Import last period cumul

In [105]:
df_last_ax1 = pd.read_excel(f'./input/{previous_period}/output_xlsxwriter.xlsx', sheet_name= 'ax1', index_col=0)

last_boost_cumul = df_last_ax1.iat[0, 7]
last_ratio_cumul = df_last_ax1.iat[0, 8]
last_mtbf_cumul = df_last_ax1.iat[0, 9]
last_mttr_cumul = df_last_ax1.iat[0, 10]
last_mtti_cumul = df_last_ax1.iat[0, 11]
last_onee_cumul = df_last_ax1.iat[0, 12]

In [7]:
df_last_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0,4.10,0.71,3.40,0,0,9989.52,4311.098067,99.901302,29.560231,10.938747,6.779662,465925.372
Indicateurs du mois 2021-05 :,0,3.69,0.24,3.45,0,0,3331.88,2141.454879,99.651680,32.724409,13.499438,8.271958,148403.387


# ONEE

In [9]:
onee_tah = pd.read_table(f'./input/{period}/CCH TAREC TR NORM {str(period_month).zfill(2)} {period_year}.XLS')
onee_nord = pd.read_table(f'./input/{period}/CCH TAREC NORD NORM {str(period_month).zfill(2)} {period_year}.XLS')

onee = onee_tah.append(onee_nord)

onee_received = onee['kWh-Rec'].sum()/6/1000

In [ ]:
onee_tah = pd.read_excel(
    f'./input/{period}/CCH TAREC TR NORM {str(period_month).zfill(2)} {period_year}.xlsx')
onee_nord = pd.read_excel(
    f'./input/{period}/CCH TAREC NORD NORM {str(period_month).zfill(2)} {period_year}.xlsx')

onee = onee_tah.append(onee_nord)

onee_received = onee['kWh-Rec'].sum()/6/1000


In [10]:
onee.groupby(['Date', 'Time']).sum()['kWh-Del'].max()


1518

In [9]:
onee_received

74786.624

In [106]:
onee_period = float(input('onee_period') or onee_received)

if period_month == 1:
    onee_cumul = onee_period
else:
    onee_cumul = (last_onee_cumul + onee_period)

In [107]:
last_onee_cumul

465925.372

# df ax 19 Performance ratio 

In [12]:

# df_ax19 = pd.read_html(f'./input/{period}/TAREC REPORTING.html', thousands=' ', decimal=',')[0]

# df_ax19['Turbine'] = df_ax19['Turbine'].str.extract(r'\[[^\d]*(\d+)[^\d]*\]').values.astype(int)

# df_ax19 = df_ax19[['Turbine', 'Ratio']]

# df_ax19.Ratio.mean()

# # last_ratio_cumul = input('last_ performance_ratio_cumul') or 94.846

# ratio_period = df_ax19.Ratio.mean()

# ratio_cumul = (last_ratio_cumul*days_to_previous_period + ratio_period*days_in_period)/(days_to_previous_period+days_in_period)

# df_ax19

# Calcul mois

In [340]:
df = pd.read_csv(f'../Availability_Warranty_Dash/monthly_data/results/{period}-Availability.csv',
                    decimal=',', sep=';')
df.columns = ['_'.join(str(v) for v in tup) if type(tup) is tuple else tup for tup in df.columns]
df.TimeStamp = pd.to_datetime(df.TimeStamp)
df.to_pickle(f'../Data/results/{period}.pkl')

In [4]:
# import pandas as pd

# for i in range(1, 6):
    
#     period = f'2021-{str(i).zfill(2)}'
#     df = pd.read_csv(f'../Availability_Warranty_Dash/monthly_data/results/{period}-Availability.csv',
#                         decimal=',', sep=';')
#     df.columns = ['_'.join(str(v) for v in tup) if type(tup) is tuple else tup for tup in df.columns]
#     df.TimeStamp = pd.to_datetime(df.TimeStamp)
#     df.to_pickle(f'../Data/results/{period}.pkl')

In [341]:
results = pd.read_pickle(f'../DATA/results/{period}.pkl')
results['StationId'] = results['StationId'] - 2307404

In [294]:
alarms = read_sum(period)
alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)
alarms['StationId'] = alarms['StationId'] - 2307404

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-06-sum.mdb;
../DATA/SUM/2021-06-sum.mdb Loaded


In [295]:
results.head(2)

,StationId,TimeStamp,RealPeriod,Period 0(s),Period 1(s),UK Text,Duration 2006(s),wtc_kWG1Tot_accum,wtc_kWG1TotE_accum,wtc_ActPower_min,...,EL_PowerRed,EL_2006,EL_wind,Duration lowind(s),EL_wind_start,Duration lowind_start(s),EL_alarm_start,Duration alarm_start(s),EL_indefini_left,EL_Misassigned
0,1,2021-06-01 00:10:00,0.0,0.0,0.0,0,0,310.0,310.0,1156.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,1,2021-06-01 00:20:00,0.0,0.0,0.0,0,0,286.0,285.0,1062.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [296]:
# abbreviations
Frame = pd.DataFrame
c1 = 'Alarmcode'
c2 = 'Error Group'

error_group = pd.concat([Frame({c1: range(901, 2101), c2: 'System'}),
                         Frame({c1: range(2101, 3000), c2: 'Generator'}),
                         Frame({c1: range(3100, 4000), c2: 'Hub'}),
                         Frame({c1: range(4100, 5000), c2: 'Gear'}),
                         Frame({c1: range(5000, 6000), c2: 'Grid'}),
                         Frame({c1: range(6100, 7000), c2: 'Rotor'}),
                         Frame({c1: range(7100, 8000), c2: 'Hydraulics'}),
                         Frame({c1: range(8000, 8400), c2: 'Environement'}),
                         Frame({c1: range(8450, 9000),
                                c2: 'Turbine cond...'}),
                         Frame({c1: range(9100, 10000), c2: 'Brake'}),
                         Frame({c1: range(10100, 11000), c2: 'Yaw'}),
                         Frame({c1: range(11100, 12000), c2: 'PFC'}),
                         Frame({c1: range(12100, 13000), c2: 'Transformer'}),
                         Frame({c1: range(13000, 14000), c2: 'Converter-1'}),
                         Frame({c1: range(14000, 15000), c2: 'Gen.inverter'}),
                         Frame({c1: range(15000, 16000), c2: 'Grid inverter'}),
                         Frame({c1: range(16000, 17000), c2: 'Main bearing'}),
                         Frame({c1: range(17000, 18300), c2: 'Converter-2'}),
                         Frame({c1: range(62001, 64000), c2: 'Controller'}),
                         Frame({c1: range(64000, 65200), c2: 'MISCELLANEOUS'})])


In [394]:
reorder = ['System',
           'Generator',
           'Hub',
           'Gear',
           'Grid',
           'Rotor',
           'Hydraulics',
           'Environement',
           'Turbine cond...',
           'Brake',
           'Yaw',
           'PFC',
           'Transformer',
           'Converter-1',
           'Gen.inverter',
           'Grid inverter',
           'Main bearing',
           'Converter-2',
           'Controller',
           'Miscellaneous']


In [298]:
error_list = pd.read_excel(
    r'Error_Type_List_Las_Update_151209.xlsx',
    usecols=lambda x: x != 'Type Selected')

error_list.Alarmcode = error_list.Alarmcode.astype(int)  # ,errors='ignore'

error_list.drop_duplicates(subset=['Alarmcode'], inplace=True)

error_list = error_list.merge(error_group, on='Alarmcode', how='left')

# ------------------------------------------------------------------------
''' label scada alarms with coresponding error type
and only keep alarm codes in error list'''
result_sum = pd.merge(alarms, error_list,
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
result_sum = result_sum.loc[result_sum['Error Type'].isin([1, 0])]

# apply cascade
alarms_result_sum = apply_cascade(result_sum)

# only keep  parent alarms
parent_result_sum = alarms_result_sum.query('TimeOn == NewTimeOn')

# dash duree
main_result_sum = alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')
# main_result_sum['ALL duree'] = main_result_sum['TimeOff'] -main_result_sum['TimeOn']

In [299]:
alarms.query('Alarmcode == 1029')

,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter


In [300]:
alarms_result_sum.query('Alarmcode == 10400')


,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter,UK Text,Error Type,Error Group,NewTimeOn,RealPeriod,Period Siemens(s),Period Tarec(s)
155,2021-06-11 01:35:53.000,2021-06-11 01:40:53.000,3,10400,4667289,...,Yaw fail waiting,1,Yaw,2021-06-11 01:35:53.000,0 days 00:05:00,0 days 00:05:00,NaT
157,2021-06-16 00:33:43.000,2021-06-16 00:38:43.000,3,10400,4684324,...,Yaw fail waiting,1,Yaw,2021-06-16 00:33:43.000,0 days 00:05:00,0 days 00:05:00,NaT
159,2021-06-16 00:47:07.000,2021-06-16 00:56:06.999,3,10400,4684428,...,Yaw fail waiting,1,Yaw,2021-06-16 00:56:06.999,0 days 00:00:00,0 days 00:00:00,NaT
160,2021-06-16 01:03:48.000,2021-06-16 01:17:48.000,3,10400,4684501,...,Yaw fail waiting,1,Yaw,2021-06-16 01:03:48.000,0 days 00:14:00,0 days 00:14:00,NaT
161,2021-06-16 01:26:39.999,2021-06-16 01:45:39.999,3,10400,4684660,...,Yaw fail waiting,1,Yaw,2021-06-16 01:26:39.999,0 days 00:19:00,0 days 00:19:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7321,2021-06-30 14:16:43.999,2021-06-30 14:29:18.000,127,10400,4744669,...,Yaw fail waiting,1,Yaw,2021-06-30 14:16:43.999,0 days 00:12:34.001000,0 days 00:12:34.001000,NaT
7322,2021-06-30 16:04:25.000,2021-06-30 16:21:25.000,127,10400,4745103,...,Yaw fail waiting,1,Yaw,2021-06-30 16:04:25.000,0 days 00:17:00,0 days 00:17:00,NaT
7323,2021-06-30 16:39:01.999,2021-06-30 17:00:00.999,127,10400,4745147,...,Yaw fail waiting,1,Yaw,2021-06-30 16:39:01.999,0 days 00:20:59,0 days 00:20:59,NaT
7324,2021-06-30 20:10:07.000,2021-06-30 20:32:08.000,127,10400,4745391,...,Yaw fail waiting,1,Yaw,2021-06-30 20:10:07.000,0 days 00:22:01,0 days 00:22:01,NaT


## Power Boost

In [301]:
def read_boost(file_name):
    usecols_cnt = '''TimeStamp, StationId, wtc_BoostKWh_endvalue'''

    sql_cnt = f"Select {usecols_cnt} FROM tblSCTurCount;"

    cnt = mdb_to_df(file_name=file_name, sql=sql_cnt)

    cnt['TimeStamp'] = pd.to_datetime(
        cnt['TimeStamp'], format='%m/%d/%y %H:%M:%S')

    return cnt

def boost_endvalue_check(df):
    df = df.reset_index().sort_values('TimeStamp')
    df.drop('index', axis=1, inplace=True)
    clmn = df['wtc_BoostKWh_endvalue']
    x = (clmn.iat[-1] - clmn.iat[0])/1000
    # df = df.diff().query('wtc_BoostKWh_endvalue < 0')
    if x < 0:
        return 0
    else:
        return x

In [302]:

# last_boost_cumul = input('last_boost_cumul') or 3458.1

boost = read_boost(f'../Availability_Warranty_Dash/monthly_data/uploads/{period}/{period}-cnt.mdb')

boost_period = boost.groupby('StationId').apply(lambda df: boost_endvalue_check(df)).sum()

if period_month == 1:
    boost_cumul = boost_period
else:
    boost_cumul = last_boost_cumul + boost_period

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../Availability_Warranty_Dash/monthly_data/uploads/2021-06/2021-06-cnt.mdb;
../Availability_Warranty_Dash/monthly_data/uploads/2021-06/2021-06-cnt.mdb Loaded


In [303]:
boost_period, boost_cumul

(1349.7322265624998, 5660.8302937316885)

## MTBF MTTR 

In [304]:
manual_stop = [1001,
               1002,
               1005,
               1007,
               1008,
               1009,
               1015,
               1016,
               1017,
               1021,
               1025,
               1026,
               1027,
               8216,
               8217,
               8222,
               ]

In [305]:
query = '(Alarmcode not in @manual_stop) & (RealPeriod >= @pd.Timedelta("2H"))'

freq_period = len(main_result_sum.query(query))

duree_period = (main_result_sum.query(query)
                               .RealPeriod
                               .sum().total_seconds()
                               / 3600)

mtbf_period = (days_in_period * 24 * 131 - duree_period) / freq_period / 24

mtbf_period

50.706639159752285

In [306]:
mttr_period = duree_period/freq_period

mttr_period

7.975725101010102

In [307]:
# Generate the new calculated MTTBF and MTTR for each error group

MTBF_MTTR_cat = (main_result_sum.query(query).groupby('Error Group')
                                .agg(duree=('RealPeriod', lambda x: x.sum().total_seconds() / 3600),
                                     freq=('TimeOn', 'count'),
                                     )
                                .rename_axis('Catégorie'))

temps_total_period = days_in_period * 24 * 131

duree_period = MTBF_MTTR_cat['duree']
freq_period = MTBF_MTTR_cat['freq']


MTBF_MTTR_cat['MTTR en h'] = duree_period/freq_period

MTBF_MTTR_cat['MTBF en j'] = (
    temps_total_period - duree_period) / (freq_period * 24)


In [308]:
MTBF_MTTR_cat

,duree,freq,MTTR en h,MTBF en j
Catégorie,,,,
Brake,29.236388,6,4.872731,654.796970
Controller,93.443888,9,10.382654,436.234056
Converter-1,138.569167,13,10.659167,301.863560
Environement,47.692223,7,6.813175,561.144689
Gear,85.935833,14,6.138274,280.458524
Hub,79.118055,8,9.889757,490.837927
Hydraulics,35.470556,8,4.433819,491.065258
Rotor,5.779722,1,5.779722,3929.759178
System,21.669167,3,7.223056,1309.699039


## MTTI

In [309]:

def mtti(df, turbine):

    # df = main_result_sum.query('StationId == @turbine')
    df = (df.loc[(df.TimeOn == df.NewTimeOn).shift(-1).fillna(True)]
            .query('TimeOn != NewTimeOn').assign(root=False)
            .append(df.query('TimeOn == NewTimeOn').assign(root=True))
            .sort_values('ID'))

    df.loc[(df.root == False).shift(-1).fillna(False), 'TimeOff'] = df.loc[(df.root == False)]['TimeOff'].values
    df['RealPeriod'] = df['TimeOff'] - df['TimeOn']

    df.drop(df.loc[df.root == False].index, inplace=True)

    df2 = alarms.query('StationId == @turbine & Alarmcode == 50000')

    df = (df2.append(df).sort_values('TimeOn').reset_index(drop=True))

    df = df.query('Alarmcode not in @manual_stop') 

    df = df.loc[(df['Alarmcode']).shift() != (df['Alarmcode'])]

    df = cascade(df)

    mask_root = (df.TimeOn.values >= df.TimeOffMax.values)
    mask_children = (df.TimeOn.values < df.TimeOffMax.values) & (
        df.TimeOff.values > df.TimeOffMax.values)
    mask_embedded = (df.TimeOff.values <= df.TimeOffMax.values)

    df.loc[mask_root, 'NewTimeOn'] = df.loc[mask_root, 'TimeOn']
    df.loc[mask_children, 'NewTimeOn'] = df.loc[mask_children, 'TimeOffMax']
    df.loc[mask_embedded, 'NewTimeOn'] = df.loc[mask_embedded, 'TimeOff']

    
    df.drop(df.query('TimeOn == NewTimeOn & Alarmcode == 50000').index, inplace=True)

    mask = (df.Alarmcode == 50000).shift(-1, fill_value=False)

    df.loc[mask, 'TimeTI'] = df.loc[(df.Alarmcode == 50000), 'TimeOn'].values

    df.loc[mask, 'TimeTI'] = df.loc[mask, 'TimeTI'] - df.loc[mask, 'TimeOn']

    df['TimeTI'] = df['TimeTI'].astype('timedelta64[ns]')


    return df


In [310]:
mtti_df = main_result_sum.groupby('StationId').apply(lambda x: mtti(x, x.name)).reset_index(drop=True)
mtti_df

,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter,UK Text,Error Type,Error Group,NewTimeOn,RealPeriod,Period Siemens(s),Period Tarec(s),root,TimeOffMax,TimeTI
0,2021-06-01 11:45:40.000,2021-06-01 11:49:10.000,1,3130,4620961,...,Pitch lubrication,0,Hub,2021-06-01 11:45:40.000,0 days 00:03:30,NaT,0 days 00:03:30,True,2021-06-01 11:45:40.000,NaT
1,2021-06-01 11:51:46.999,2021-06-01 11:52:49.000,1,13902,4620994,...,"Converter tripped, Auto Start",1,Converter-1,2021-06-01 11:51:46.999,0 days 00:01:02.001000,0 days 00:01:02.001000,NaT,True,2021-06-01 11:49:10.000,NaT
2,2021-06-06 23:25:49.999,2021-06-06 23:29:19.999,1,3130,4638856,...,Pitch lubrication,0,Hub,2021-06-06 23:25:49.999,0 days 00:03:30,NaT,0 days 00:03:30,True,2021-06-01 11:52:49.000,NaT
3,2021-06-15 00:05:45.999,2021-06-15 00:09:16.000,1,3130,4678991,...,Pitch lubrication,0,Hub,2021-06-15 00:05:45.999,0 days 00:03:30.001000,NaT,0 days 00:03:30.001000,True,2021-06-11 08:57:10.076,NaT
4,2021-06-01 08:26:32.999,2021-06-01 08:30:03.000,2,3130,4618856,...,Pitch lubrication,0,Hub,2021-06-01 08:26:32.999,0 days 00:03:30.001000,NaT,0 days 00:03:30.001000,True,2021-06-01 08:26:32.999,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,2021-06-26 10:39:28.000,2021-06-26 10:50:56.999,129,7105,4734549,...,"Low pitch oil pressure, start",1,Hydraulics,2021-06-26 10:39:28.000,0 days 00:11:28.999000,0 days 00:11:27.999000,NaT,True,2021-06-14 22:17:07.999,NaT
854,2021-06-01 08:54:44.000,2021-06-01 08:58:13.999,130,3130,4619188,...,Pitch lubrication,0,Hub,2021-06-01 08:54:44.000,0 days 00:03:29.999000,NaT,0 days 00:03:29.999000,True,2021-06-01 08:54:44.000,NaT
855,2021-06-01 08:55:40.999,2021-06-01 08:59:11.000,131,3130,4619208,...,Pitch lubrication,0,Hub,2021-06-01 08:55:40.999,0 days 00:03:30.001000,NaT,0 days 00:03:30.001000,True,2021-06-01 08:55:40.999,NaT
856,2021-06-01 09:01:51.000,2021-06-01 09:02:52.999,131,13902,4619307,...,"Converter tripped, Auto Start",1,Converter-1,2021-06-01 09:01:51.000,0 days 00:01:01.999000,0 days 00:01:01.999000,NaT,True,2021-06-01 08:59:11.000,NaT


In [311]:
mtti_period = (mtti_df['TimeTI'].sum() / len(mtti_df.query('Alarmcode == 50000'))).total_seconds()/3600

mtti_period

4.233735838888889

## Graphe 3 ax5

In [312]:
df_ax5 = (main_result_sum.groupby('Error Group')
                  .agg(Freq=('Alarmcode', 'count'),
                       Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                  .reindex(reorder)
                  .dropna()
                  .reset_index()
                  )

# df_ax5.plot(kind='bar', x='Error Group', ax=ax5)
df_ax5

,Error Group,Freq,Durée
0,System,692.0,2326.290553
1,Generator,4.0,1.608611
2,Hub,677.0,136.613604
3,Gear,71.0,111.234165
4,Grid,3.0,0.000834
5,Rotor,6.0,6.197222
6,Hydraulics,140.0,56.959441
7,Environement,96.0,66.677226
8,Turbine cond...,1.0,12.568056
9,Brake,47.0,35.662222


## Graphe 4 ax6

In [313]:
df_ax6 = (main_result_sum.groupby('Alarmcode')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod',
                               lambda x: x.sum().total_seconds()/3600))
                .sort_values('Durée', ascending=False)
                .head(20)
                .reset_index()
                .sort_values('Durée', ascending=False))

# df_ax6.plot(kind='bar', x='Alarmcode', ax=ax6)
df_ax6

,Alarmcode,Freq,Durée
0,1007,129,1235.949433
1,1001,405,893.924449
2,13902,152,129.230001
3,1026,8,93.493056
4,3173,26,65.891112
5,1025,98,53.418335
6,10100,17,48.515277
7,8160,51,43.369169
8,4106,21,43.081387
9,3130,625,39.094437


## ax7

In [314]:
most_categories = (main_result_sum.groupby('Error Group')
                                  .agg({'RealPeriod': np.sum})
                                  .sort_values('RealPeriod', ascending=False)
                                  .index.values)

realperiod_sum = main_result_sum.RealPeriod.sum()
percentage_per_category = (main_result_sum.groupby('Error Group')
                                  .agg(percentage=('RealPeriod',
                                                   lambda x: 100 * x.sum() / realperiod_sum))
                                  .sort_values('percentage', ascending=False))


df_most_categories = main_result_sum.loc[main_result_sum['Error Group'].isin(most_categories)].copy()

df_most_categories['Error Group'] = df_most_categories['Error Group'].astype("category")

df_most_categories['Error Group'].cat.set_categories(most_categories, inplace=True)

df_most_categories.sort_values('Error Group', inplace=True)


# ------------------------------------------------------------------------------------------------
def extract_important_alarms(df):

    df = df.groupby('Alarmcode').agg({'RealPeriod': np.sum}).reset_index()
    
    df['percentage'] = 100 * df['RealPeriod'] / df['RealPeriod'].sum()
    df.sort_values('percentage', ascending=False, inplace=True)
    df['percentage roll'] = df['percentage'].cumsum()
    df.set_index('Alarmcode', inplace=True)


#     for i in range(1,100):
#         if len(df.query('`percentage roll` < @i')) < 3:
#             continue
#         else:
#             df = df.query('`percentage roll` < @i')
#         break

    return df

# ------------------------------------------------------------------------------------------------

df_ax7 = (df_most_categories.groupby('Error Group', sort=False)
                            .apply(lambda x: extract_important_alarms(x))
                            .reset_index()
                            .merge(error_list[['UK Text', 'Alarmcode']], how='left')
         )

# df_ax7 = (df_most_categories.groupby('Error Group')
#                              .apply(lambda x: (x.groupby('Alarmcode')
#                                                 .agg({'RealPeriod': np.sum})
#                                                 .sort_values('RealPeriod')
#                                                 .tail(4)))
#                              .reset_index()
#                              .merge(error_list[['UK Text', 'Alarmcode']]))

# ------------------------------------------------------------------------------------------------
ax7 = ''
for cat in most_categories:
    
    str1_ax7 = ''
    
    for i, row in df_ax7.query('`Error Group` == @cat').iterrows():
        
        Alarmcode = row['Alarmcode']
        UK_Text = row['UK Text']
        Percentage = round(row['percentage'], 1)
                       
        str1_ax7 += f'\n\t-\"{Alarmcode} - {UK_Text} - {Percentage}% \".'
        
    
    str2_ax7 = f"""
La catégorie {cat} - {round(percentage_per_category.loc[cat, 'percentage'], 2)}%, les alarmes les plus importantes sont:{str1_ax7}
    """
    ax7 += str2_ax7
    
worst_mtbf_cat = MTBF_MTTR_cat['MTBF en j'].idxmin()
worst_mtbf = int(MTBF_MTTR_cat['MTBF en j'].min())

ax7 += f'\nLe MTBF le moins bon du mois est celui de la catégorie "{worst_mtbf_cat}" :  {worst_mtbf} jours '


In [315]:
MTBF_MTTR_cat

,duree,freq,MTTR en h,MTBF en j
Catégorie,,,,
Brake,29.236388,6,4.872731,654.796970
Controller,93.443888,9,10.382654,436.234056
Converter-1,138.569167,13,10.659167,301.863560
Environement,47.692223,7,6.813175,561.144689
Gear,85.935833,14,6.138274,280.458524
Hub,79.118055,8,9.889757,490.837927
Hydraulics,35.470556,8,4.433819,491.065258
Rotor,5.779722,1,5.779722,3929.759178
System,21.669167,3,7.223056,1309.699039


In [32]:
# with open("Output.txt", "w", encoding='UTF-8') as text_file:
#     text_file.write(ax7)

## Graph 6 ax9

In [342]:
df_ax9 = pd.merge(
        (results.groupby('StationId')
                       .agg(**{'Durée arrêts': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .sort_values('Durée arrêts', ascending=False)
                 # .head(25)
                 .reset_index()
         ),
         (alarms.groupby('StationId')
                      .agg(**{'Freq arrêts': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
         on='StationId'
        ).sort_values('Durée arrêts', ascending=False).head(20)


df_ax9['Duration_20_25 reel'] = df_ax9['Durée autres']
df_ax9['Durée autres'] = (df_ax9['Duration_20_25 reel'] - df_ax9['Durée arrêts'])
# df_ax9.plot(kind='bar', x='StationId', ax=ax9)

df_ax9

,StationId,Durée arrêts,Durée autres,Freq arrêts,Freq autres,Duration_20_25 reel
0,96,176.769722,0.947778,20,11,177.717500
1,107,122.020556,12.585556,101,58,134.606111
2,36,61.183056,2.233333,45,16,63.416389
3,45,61.056111,2.353056,20,21,63.409167
4,120,59.728889,2.323889,13,16,62.052778
5,63,57.845556,2.164722,30,29,60.010278
6,48,55.842222,1.688056,24,16,57.530278
7,75,52.245556,3.600278,20,25,55.845833
8,77,49.876111,4.973056,58,49,54.849167
9,127,48.076944,3.722222,39,36,51.799167


In [383]:
df = alarms.query('Alarmcode == 13902').copy()

df['Duree'] = (df['TimeOff'] - df['TimeOn']).dt.total_seconds()


df.sort_values('Duree', ascending=False).head(15).groupby(
    'StationNr').sum().sort_values('Duree', ascending=False)[['Duree']]/3600


,Duree
StationNr,
2307440,35.593056
2307524,19.193333
2307468,18.942222
2307406,15.334444
2307425,14.280278
2307435,10.005556
2307522,9.764167
2307444,7.723611
2307525,7.433889


In [386]:
df = alarms.query('Alarmcode == 63036').copy()

df['Duree'] = (df['TimeOff'] - df['TimeOn']).dt.total_seconds()


df.sort_values('Duree', ascending=False).head(15).groupby(
    'StationNr').sum().sort_values('Duree', ascending=False)[['Duree']]/3600


,Duree
StationNr,
2307449,23.140556
2307460,1.473889
2307437,1.100278
2307430,0.956944
2307439,0.568889
2307438,0.304444
2307442,0.184722
2307434,0.173333
2307436,0.077223


In [384]:
df = alarms.query('Alarmcode == 3173').copy()

df['Duree'] = (df['TimeOff'] - df['TimeOn']).dt.total_seconds()


df.sort_values('Duree', ascending=False).head(15).groupby(
    'StationNr').sum().sort_values('Duree', ascending=False)[['Duree']]/3600


,Duree
StationNr,
2307511,63.847222
2307496,0.413056


,Duree
StationNr,
2307440,35.593056
2307524,19.193333
2307468,18.942222
2307406,15.334444
2307425,14.280278
2307435,10.005556
2307522,9.764167
2307444,7.723611
2307525,7.433889


## Graphe 7 ax10

In [129]:
df_ax10 = (results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('ELNX', ascending=False)
                .reset_index())

df_ax10[['ELNX', 'EL_indefini_left']] = df_ax10[['ELNX', 'EL_indefini_left']] / 1e3

df_ax10 = round(df_ax10, 2).abs().head(20)

df_ax10.rename({'ELNX': 'Energie perdue SGRE'}, inplace=True, axis=1)

df_ax10

,StationId,Energie perdue SGRE,EL_indefini_left
0,96,256.96,0.00
1,107,172.06,2.75
2,63,99.77,0.00
3,45,99.56,0.00
4,36,97.40,0.00
5,127,76.63,0.00
6,95,67.55,0.00
7,59,64.04,0.00
8,120,63.99,0.00
9,48,56.90,0.00


In [334]:
results.query('StationId == 32 & EL_indefini_left > 0')

,StationId,TimeStamp,RealPeriod,Period 0(s),Period 1(s),UK Text,Duration 2006(s),wtc_kWG1Tot_accum,wtc_kWG1TotE_accum,wtc_ActPower_min,...,EL_PowerRed,EL_2006,EL_wind,Duration lowind(s),EL_wind_start,Duration lowind_start(s),EL_alarm_start,Duration alarm_start(s),EL_indefini_left,EL_Misassigned
135022,32,2021-06-08 15:50:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,278.75,0.0
135023,32,2021-06-08 16:00:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,310.25,0.0
135024,32,2021-06-08 16:10:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,324.37,0.0
135025,32,2021-06-08 16:20:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,342.29,0.0
135026,32,2021-06-08 16:30:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,327.60,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136101,32,2021-06-16 03:40:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,81.29,0.0
136104,32,2021-06-16 04:10:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,74.97,0.0
136105,32,2021-06-16 04:20:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,62.86,0.0
136112,32,2021-06-16 05:30:00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,57.24,0.0


## ax11

In [130]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

period_month_name_ascii = remove_accents(period_month_name).upper()

### Convert canvas after manual tidying

In [346]:
from openpyxl import load_workbook
import openpyxl

filename = f'./input/{period}/PART REPLACED SGRE SERVICE {period_year}.xlsx'

wb = load_workbook(filename=filename)

ws = wb[period_month_name_ascii]

# ws.unmerge_cells('A2:D2')

parts = pd.DataFrame(ws.values)


merged_ranges = [np.subtract(np.array(openpyxl.utils.cell.range_boundaries(str(i))), [1, 1, 0, 0])
                 for i in ws.merged_cells.ranges]


for single_range in merged_ranges:

    parts.iloc[(single_range[1]):(single_range[3]),
            (single_range[0]):(single_range[2])].ffill(inplace=True)


new_header = parts.iloc[7]  # grab the first row for the header
parts = parts[8:]  # take the data less the header row
parts.columns = new_header.rename(None)  # set the header row as the df header

parts.dropna(subset=['DESCRIPTION'], inplace=True)

parts = parts.assign(TURBINE=parts.TURBINE.astype(
    'str').str.split('/')).explode('TURBINE')

parts.reset_index(drop=True, inplace=True)

parts.rename(
    columns={'DESCRIPTION': 'REFERENCE',
             'DEFECT': 'DESIGNATION',
             'QTE': 'QTE SORTIE'},
    inplace=True)

parts['REFERENCE'] = parts['REFERENCE'].str.upper()

parts[['DESIGNATION', 'TURBINE']] = parts[['DESIGNATION', 'TURBINE']].apply(
    lambda s: s.str.upper().str.lstrip('\t'), 1)

parts = parts.astype({'TURBINE': int, 'QTE SORTIE': float})

parts


,TURBINE,SOLVED ERROR/WORK DONE,REFERENCE,DESIGNATION,QTE SORTIE
0,12,TMSE/Gen Align/Lift/Crane Inspections (6th Yea...,A9B10044880,BRACKET FOR AVANTILADDER/FOUNDATION,2.0
1,30,TMSE/Gen Align/Lift/Crane Inspections (6th Yea...,A9B10044880,BRACKET FOR AVANTILADDER/FOUNDATION,2.0
2,124,Repalce defect Yaw Motor,A9B10091496,CONTACTOR FOR (AF16-30-10-13) 3P 100-250V,2.0
3,124,Repalce defect Yaw Motor,A9B10055151,YAW MOTOR,1.0
4,75,Instal Sonic & defect transducer,A9B00019621,ANEMOMETRE,1.0
...,...,...,...,...,...
170,56,UPS Batt replacement,A9B10011190,BATTERY PACK (8 PC) FOR 809.C1 OR 809.C2,2.0
171,130,Sonic Installation,A9B00019620,WINDVANE,2.0
172,131,Sonic Installation,A9B00019620,WINDVANE,2.0
173,130,Sonic Installation,A9B00019621,ANEMOMETRE,2.0


In [ ]:
parts = pd.read_excel(
    f'./input/{period}/PART REPLACED SGRE SERVICE {period_year}.xlsx', header=7, sheet_name=period_month_name_ascii)


In [347]:

df_ax11_last = pd.read_excel(f'./results/{previous_period}/TB Performance {previous_period}.xlsx', sheet_name='Feuil3', skiprows=1,
                       usecols='C:E')


df_ax11 = parts.rename(
    columns={'REFERENCE': 'item', 'QTE SORTIE': period_month_name})

df_ax11 = df_ax11.groupby('item').sum().reset_index()[['item', period_month_name]]

# if period_month == 1:

#     df_ax11['Year'] = df_ax11[period_month_name].fillna(0)
#     df_ax11 = df_ax11[['item', 'Year', period_month_name]]
    
# else:
df_ax11 = df_ax11_last.merge(df_ax11, how='outer')[['item', 'Year', period_month_name]]
df_ax11['Year'] = (df_ax11['Year'].fillna(0) + df_ax11[period_month_name].fillna(0))

df_ax11 = df_ax11.fillna(0).astype(int, errors='ignore')

df_ax11


,item,Year,Juin
0,A9B10055151,560,38
1,A9B00573133,160,5
2,A9B10027591,48,0
3,A9B00075055,159,0
4,A9B00019620,87,42
...,...,...,...
185,A9B00026254,1,1
186,A9B0008164,2,2
187,A9B00090421,1,1
188,A9B10007824,1,1


## ax12

In [ ]:
df_ax12 = pd.read_excel(f'./input/{period}/PART REPLACED SGRE SERVICE {period_year}.xlsx',
                        skiprows=7, sheet_name=period_month_name_ascii)


In [162]:
df_ax12 = parts.copy()

df_ax12['TURBINE'] = df_ax12['TURBINE'].astype(str)

df_ax12['DESIGNATION'] = df_ax12['DESIGNATION'].str.upper().str.lstrip('\t')

df_ax12 = (df_ax12.groupby('DESIGNATION')
                  .agg({'QTE SORTIE': np.sum,
                        'TURBINE': lambda x: set(map(int, x.str.upper().str.lstrip('T0').str.cat(sep=' ').split()))})
                  .reset_index())

max_turbines = set(df_ax10.StationId.astype(int))
df_ax12['max_turbines'] = df_ax12['TURBINE'].apply(lambda x: x & max_turbines)

df_ax12.sort_values('QTE SORTIE').reset_index(drop=True).to_clipboard()

df_ax12


,DESIGNATION,QTE SORTIE,TURBINE,max_turbines
0,ANEMOMETRE,50.0,"{128, 130, 131, 5, 6, 9, 11, 16, 30, 31, 48, 6...","{75, 11, 77, 107, 48, 120, 30, 127}"
1,BATTERY PACK (8 PC) FOR 809.C1 OR 809.C2,13.0,"{39, 47, 52, 56, 58, 59, 125}","{58, 59}"
2,BRACKET FOR AVANTILADDER/FOUNDATION,34.0,"{32, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 27...","{32, 2, 11, 30}"
3,CIRCULATION PUMP,1.0,{40},{}
4,CONTACTOR (AF12-30-10-13) 3P 100-250V,4.0,"{29, 111}",{}
5,CONTACTOR FOR (AF16-30-10-13) 3P 100-250V,2.0,{124},{}
6,COUPLING MOTOR CAST IRON 38/45 38H7,4.0,"{45, 107, 92, 93}","{107, 45}"
7,COUPLING PUMP 38/45.25H7 CAST IRON,4.0,"{45, 107, 92, 93}","{107, 45}"
8,EL-MOTOR 3X690V/50HZ/1.3KW RPM 3000 >,1.0,{88},{}
9,FAN A3,1.0,{45},{45}


In [51]:
max_turbines

{1,
 13,
 36,
 41,
 45,
 46,
 48,
 49,
 50,
 51,
 59,
 60,
 68,
 74,
 75,
 93,
 95,
 105,
 107,
 121}

In [351]:
df_ax12 = parts.copy()

df_ax12 = df_ax12.groupby(['DESIGNATION', 'TURBINE']).sum(
).sort_values(['DESIGNATION', 'QTE SORTIE'])

df_ax12


QTE SORTIE
DESIGNATION TURBINE            
ANEMOMETRE  11              1.0
            31              1.0
            75              1.0
            77              1.0
            85              1.0
...                         ...
YAW MOTOR   30              4.0
            48              4.0
            82              4.0
            88              4.0
            77              5.0

[148 rows x 1 columns]

In [36]:
parts_replaced


7,TURBINE,SOLVED ERROR/WORK DONE,REFERENCE,DESIGNATION,QTE SORTIE
0,12,TMSE/Gen Align/Lift/Crane Inspections (6th Yea...,A9B10044880,BRACKET FOR AVANTILADDER/FOUNDATION,2
1,30,TMSE/Gen Align/Lift/Crane Inspections (6th Yea...,A9B10044880,BRACKET FOR AVANTILADDER/FOUNDATION,2
2,124,Repalce defect Yaw Motor,A9B10091496,CONTACTOR FOR (AF16-30-10-13) 3P 100-250V,2
3,124,Repalce defect Yaw Motor,A9B10055151,YAW MOTOR,1
4,75,Instal Sonic & defect transducer,A9B00019621,ANEMOMETRE,1
...,...,...,...,...,...
170,56,UPS Batt replacement,A9B10011190,BATTERY PACK (8 PC) FOR 809.C1 OR 809.C2,2
171,130,Sonic Installation,A9B00019620,WINDVANE,2
172,131,Sonic Installation,A9B00019620,WINDVANE,2
173,130,Sonic Installation,A9B00019621,ANEMOMETRE,2


## table 1 ax1 row 2

In [165]:
df_ax1 = pd.DataFrame(columns=['LTA-Lost Time', 'Indispo. Total %','Indispo. Tarec %', 'Indispo. Siemens %', 'Indispo. ONEE %', 'Indispo. Ebop %',
                               'Pertes élctriques en MWh', 'Power Boost en MWh', 'Performance moyenne des turbines',
                               'MTBF - Mean Time Between Failure', 'MTTR - Mean Time To Repair', 'MTTI - Mean Time To Intervention',
                               'Compteurs ONEE MWh'],
                     index=['Indicateurs annuels :', f'Indicateurs du mois {period} :'])
df_ax1 = df_ax1.fillna(0.)

In [166]:
df_ax1.iat[1, 1] = round(100*results['RealPeriod'].sum()/3600/24/131/days_in_period,2)
df_ax1.iat[1, 2] = round(100*results['Period 0(s)'].sum()/3600/24/131/days_in_period,2)
df_ax1.iat[1, 3] = round(100*results['Period 1(s)'].sum()/3600/24/131/days_in_period,2)

df_ax1.iat[1, 6] = round(results.wtc_kWG1TotE_accum.sum()/1000 - onee_period,2)

df_ax1.iat[1, 7] = boost_period

# df_ax1.iat[1, 8] = ratio_period
#
df_ax1.iat[1, 9]  = mtbf_period
df_ax1.iat[1, 10] = mttr_period
df_ax1.iat[1, 11] = mtti_period
df_ax1.iat[1, 12] = onee_period

In [167]:
round(results.wtc_kWG1TotE_accum.sum()/1000 - onee_period,2)


2502.41

In [168]:
df_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0
Indicateurs du mois 2021-06 :,0.0,3.35,1.45,1.9,0.0,0.0,2502.41,1349.732227,0.0,50.706639,7.975725,4.233736,141075.1


## table 2 ax2

In [169]:

df_ax2_header = ['', '', '', 'Travaux de maintenance']
df_ax2 = pd.DataFrame(columns=['Transformateur', 'Serial Defect', 'Corrosion', 'Inspection Fin de garantie',
                               'FSA', 'Assurance', 'Conformité DNSSI', 'Main Bearing', 'BAX', 'Darwin',
                               'Procédure de securité', 'SMI', 'Exploitation', 'Mesures', 'Audit ENGIE'],
                     index=['Etape terminée', 'Etape en cours'])

In [170]:
df_ax2

,Transformateur,Serial Defect,Corrosion,Inspection Fin de garantie,FSA,Assurance,Conformité DNSSI,Main Bearing,BAX,Darwin,Procédure de securité,SMI,Exploitation,Mesures,Audit ENGIE
Etape terminée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Etape en cours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Calcul Cumul

In [345]:
cumul_alarms = pd.DataFrame()
cumul_results = pd.DataFrame()

for month in range(1, period_month+1):

    period_cumul = f'{str(period_year).zfill(2)}-{str(month).zfill(2)}'
    alarms = read_sum(f'{period_cumul}')

    cumul_alarms = pd.concat([cumul_alarms, alarms])

    # -------------------------------------------------------------------------
    results = pd.read_pickle(f'../DATA/results/{period_cumul}.pkl')
    
    # results = results[['StationId', 'ELNX', 'Duration 115(s)', 'Duration 20-25(s)',
    #                    'Period 0(s)', 'Period 1(s)', 'RealPeriod',
    #                    'EL_indefini_left']]

    cumul_results = pd.concat([cumul_results, results])


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-01-sum.mdb;
../DATA/SUM/2021-01-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-02-sum.mdb;
../DATA/SUM/2021-02-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-03-sum.mdb;
../DATA/SUM/2021-03-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-04-sum.mdb;
../DATA/SUM/2021-04-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-05-sum.mdb;
../DATA/SUM/2021-05-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-06-sum.mdb;
../DATA/SUM/2021-06-sum.mdb Loaded


In [177]:
cumul_alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)

# only run once otherwise negative turbine numbers
cumul_alarms['StationId'] = cumul_alarms['StationId'] - 2307404
cumul_results['StationId'] = cumul_results['StationId'] - 2307404
results['StationId'] = results['StationId'] - 2307404

In [178]:
cumul_result_sum = pd.merge(cumul_alarms, error_list[[
                        'Alarmcode', 'Error Type', 'Error Group']],
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
cumul_result_sum = cumul_result_sum.loc[cumul_result_sum['Error Type'].isin([1, 0])]

# apply cascade
cumul_alarms_result_sum = apply_cascade(cumul_result_sum)

#only keep  parent alarms
cumul_parent_result_sum = cumul_alarms_result_sum.query('TimeOn	 == NewTimeOn')
  
cumul_main_result_sum = cumul_alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')

## MTTI MTTR MTBF cumulé

In [179]:

mtbf_cumul = (last_mtbf_cumul*days_to_previous_period + mtbf_period *
              days_in_period)/(days_to_previous_period+days_in_period)

mttr_cumul = (last_mttr_cumul*days_to_previous_period + mttr_period *
              days_in_period)/(days_to_previous_period+days_in_period)

mtti_cumul = (last_mtti_cumul*days_to_previous_period + mtti_period *
              days_in_period)/(days_to_previous_period+days_in_period)

## Table 2 ax1 row 1

In [180]:
nbr_jrs_total = len(pd.date_range(f'{period_year}-01-01', f'{lastday_period}', freq='D'))

df_ax1.iat[0, 1] = round(100*cumul_results['RealPeriod'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 2] = round(100*cumul_results['Period 0(s)'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 3] = round(100*cumul_results['Period 1(s)'].sum()/3600/24/131/nbr_jrs_total,2)

df_ax1.iat[0, 6] = round(cumul_results.wtc_kWG1TotE_accum.sum()/1000 - onee_cumul,2)

df_ax1.iat[0, 7] = boost_cumul

# df_ax1.iat[0, 8] = ratio_cumul
# 
df_ax1.iat[0, 9] =  mtbf_cumul
df_ax1.iat[0, 10] = mttr_cumul
df_ax1.iat[0, 11] = mtti_cumul
df_ax1.iat[0, 12] = onee_cumul

In [181]:
100*round(cumul_results.wtc_kWG1TotE_accum.sum()/1000 - onee_cumul,2)/(cumul_results.wtc_kWG1TotE_accum.sum()/1000)

2.0164783231675534

In [182]:
df_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0.0,3.98,0.83,3.15,0.0,0.0,12491.93,5660.830294,0.0,33.065161,10.447638,6.357685,607000.472
Indicateurs du mois 2021-06 :,0.0,3.35,1.45,1.90,0.0,0.0,2502.41,1349.732227,0.0,50.706639,7.975725,4.233736,141075.100


### Yassine Hakim

In [389]:
nbr_jrs_total * 131


23711

In [339]:
nbr_jrs_total = len(pd.date_range(
    f'{period_year}-01-01', f'{lastday_period}', freq='D'))


hakim_df = pd.DataFrame(index=[period])

hakim_df['Indispo. TAREC temps (%)'] = round(100*results['Period 0(s)'].sum()/3600/24/131/days_in_period, 2)
hakim_df['Indispo. SGRE temps (%)'] = round(100*results['Period 1(s)'].sum()/3600/24/131/days_in_period, 2)
hakim_df['Indispo. TAREC énergie (%)'] = round(100 * ELX_period /(Ep_period + ELX_period + ELNX_period + EL_2006_period + EL_PowerRed_period), 2)
hakim_df['Indispo. SGRE énergie (%)'] = round(100 * (ELNX_period + EL_2006_period + EL_PowerRed_period) /(Ep_period + ELX_period + ELNX_period + EL_2006_period + EL_PowerRed_period), 2)

hakim_df['Indispo. TAREC énergie (MWh)'] = round(ELX_period / 1e3, 2)
hakim_df['Indispo. SGRE énergie (MWh)'] = round(
    (ELNX_period + EL_2006_period + EL_PowerRed_period) / 1e3, 2)

hakim_df['Pertes électriques (MWh)'] = round(results.wtc_kWG1TotE_accum.sum()/1e3 - onee_period, 2),
hakim_df['Vent moyen Mâts (m/s)'] = round(results[['met_WindSpeedRot_mean_38',
                                                 'met_WindSpeedRot_mean_39', 'met_WindSpeedRot_mean_246']].mean().mean(), 2)

hakim_df


,Indispo. TAREC temps (%),Indispo. SGRE temps (%),Indispo. TAREC énergie (%),Indispo. SGRE énergie (%),Indispo. TAREC énergie (MWh),Indispo. SGRE énergie (MWh),Pertes électriques (MWh),Vent moyen Mâts (m/s)
2021-06,1.45,1.9,0.19,2.06,282.9,3027.93,2502.41,9.66


In [8]:
cumul_results[['TimeStamp', 'ELX', 'ELNX']].groupby(cumul_results.TimeStamp.dt.month).sum()


,ELX,ELNX
TimeStamp,,
1,176540.453482,2.156876e+06
2,142502.102937,4.253592e+06
3,619997.037936,3.987096e+06
4,684734.037690,2.227006e+06
5,232297.379581,5.660947e+06
6,0.000000,4.752800e+02


## Graphe 1 ax3

In [390]:
df_ax3 = (cumul_main_result_sum.groupby('Error Group')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                .sort_values('Freq', ascending=False)
                .reindex(reorder)
                .dropna()
                .reset_index()
                )

# df_ax3.plot(kind='bar', x='Error Group', ax=ax3)
df_ax3

,Error Group,Freq,Durée
0,System,3409.0,13735.788531
1,Generator,21.0,10.406388
2,Hub,4389.0,1020.486928
3,Gear,320.0,952.839720
4,Grid,179.0,141.079439
5,Rotor,41.0,50.811388
6,Hydraulics,1633.0,1481.202777
7,Environement,500.0,446.564451
8,Turbine cond...,2.0,15.462501
9,Brake,908.0,456.184076


## Graphe 2 ax4

In [391]:
MTBF_MTTR_cat = (main_result_sum.query(query).groupby('Error Group')
                                .agg(duree=('RealPeriod', lambda x: x.sum().total_seconds() / 3600),
                                     freq=('TimeOn', 'count'),
                                     )
                                .rename_axis('Catégorie'))

In [392]:
MTBF_MTTR_cat

,duree,freq
Catégorie,,
Brake,29.236388,6
Controller,93.443888,9
Converter-1,138.569167,13
Environement,47.692223,7
Gear,85.935833,14
Hub,79.118055,8
Hydraulics,35.470556,8
Rotor,5.779722,1
System,21.669167,3


In [186]:
MTBF_MTTR_cat_prev_cumul = pd.read_excel(
    f'../TB Performance/input/{previous_period}/output_xlsxwriter.xlsx', sheet_name='MTBF & MTTR cumul')

MTBF_MTTR_cat_prev_cumul.dropna(
    how='all', subset=['MTBF en j', 'MTTR en h'], inplace=True)

MTBF_MTTR_cat_prev_cumul.rename(
    columns={'Catégorie Alarme': 'Catégorie'}, inplace=True)

MTBF_MTTR_cat_prev_cumul.set_index('Catégorie', inplace=True)

MTBF_MTTR_cat_prev_cumul

,duree,freq,MTTR en h,MTBF en j
Catégorie,,,,
Brake,1057.710688,110,9.615552,179.426625
Controller,3484.461227,287,12.140980,68.417471
Converter,3516.499571,301,11.682723,65.230828
Environement,469.323889,57,8.233752,346.692015
Gear,1721.098887,211,8.156867,93.408946
Grid,2749.280006,267,10.296929,73.657104
Hub,2236.747221,159,14.067593,123.822655
Hydraulics,3868.442501,480,8.059255,40.874614
MISCELLANEOUS,322.473611,28,11.516915,705.984414


In [187]:
MTBF_MTTR_cat_cumul = MTBF_MTTR_cat.merge(MTBF_MTTR_cat_prev_cumul, on='Catégorie', how='outer').fillna(0)

MTBF_MTTR_cat_cumul.loc['Converter-1'] = MTBF_MTTR_cat_cumul.loc[['Converter-1', 'Converter']].sum()

MTBF_MTTR_cat_cumul = (MTBF_MTTR_cat_cumul.drop('Converter')
                                          .rename(index={'Converter-1': 'Converter'}))

MTBF_MTTR_cat_cumul

,duree_x,freq_x,duree_y,freq_y,MTTR en h,MTBF en j
Catégorie,,,,,,
Brake,29.236388,6.0,1057.710688,110.0,9.615552,179.426625
Controller,93.443888,9.0,3484.461227,287.0,12.140980,68.417471
Converter,138.569167,13.0,3516.499571,301.0,11.682723,65.230828
Environement,47.692223,7.0,469.323889,57.0,8.233752,346.692015
Gear,85.935833,14.0,1721.098887,211.0,8.156867,93.408946
Hub,79.118055,8.0,2236.747221,159.0,14.067593,123.822655
Hydraulics,35.470556,8.0,3868.442501,480.0,8.059255,40.874614
Rotor,5.779722,1.0,221.432500,29.0,7.635603,681.785298
System,21.669167,3.0,2337.491013,233.0,10.032150,84.478989


In [188]:
# Generate the new calculated MTTBF and MTTR for each error group

temps_total_cumul = len(pd.date_range(
    f'{period_year}-01-01', f'{lastday_period}', freq='D'))*24*131

MTBF_MTTR_cat_cumul['duree_y'] = MTBF_MTTR_cat_cumul['duree_x'] + MTBF_MTTR_cat_cumul['duree_y']

duree_cumul = MTBF_MTTR_cat_cumul['duree_y']

MTBF_MTTR_cat_cumul['freq_y'] = MTBF_MTTR_cat_cumul['freq_x'] + MTBF_MTTR_cat_cumul['freq_y']

freq_cumul = MTBF_MTTR_cat_cumul['freq_y']

MTBF_MTTR_cat_cumul['MTTR en h'] = duree_cumul/freq_cumul

MTBF_MTTR_cat_cumul['MTBF en j'] = (
    temps_total_cumul - duree_cumul) / (freq_cumul * 24)


In [189]:
MTBF_MTTR_cat_cumul = MTBF_MTTR_cat_cumul[['duree_y', 'freq_y', 'MTTR en h', 'MTBF en j']].reset_index(
).rename(columns={'duree_y': 'duree', 'freq_y': 'freq'})


In [190]:
MTBF_MTTR_cat_cumul

,Catégorie,duree,freq,MTTR en h,MTBF en j
0,Brake,1086.947076,116.0,9.370233,204.014746
1,Controller,3577.905115,296.0,12.087517,79.601083
2,Converter,3655.068738,314.0,11.640346,75.027724
3,Environement,517.016113,64.0,8.078377,370.147776
4,Gear,1807.034720,225.0,8.031265,105.047586
5,Hub,2315.865276,167.0,13.867457,141.404225
6,Hydraulics,3903.913056,488.0,7.999822,48.254789
7,Rotor,227.212222,30.0,7.573741,790.051094
8,System,2359.160179,236.0,9.996441,100.053821
9,Turbine cond...,15.462501,2.0,7.731250,11855.177865


In [191]:
df_ax4 = MTBF_MTTR_cat_cumul[['Catégorie', 'MTTR en h', 'MTBF en j']]
# df_ax4.rename(columns={'MTTR en h': 'MTTR', 'MTBF en j': 'MTBF'}, inplace=True)

In [192]:
df_ax4

,Catégorie,MTTR en h,MTBF en j
0,Brake,9.370233,204.014746
1,Controller,12.087517,79.601083
2,Converter,11.640346,75.027724
3,Environement,8.078377,370.147776
4,Gear,8.031265,105.047586
5,Hub,13.867457,141.404225
6,Hydraulics,7.999822,48.254789
7,Rotor,7.573741,790.051094
8,System,9.996441,100.053821
9,Turbine cond...,7.731250,11855.177865


## Graphe 5 ax8

In [193]:
df_ax8 = pd.merge(
        (cumul_results.groupby('StationId')
                       .agg(**{'Durée arrêts': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .reset_index()
         ),
         (cumul_alarms.groupby('StationId')
                      .agg(**{'Freq arrêts': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
        ).sort_values('Durée arrêts', ascending=False).head(20)

# df_ax8.plot(kind='bar', x='StationId', ax=ax8)

# for stacked bar excel chart
df_ax8['Duration_20_25 reel'] = df_ax8['Durée autres']
df_ax8['Durée autres'] = df_ax8['Duration_20_25 reel'] - df_ax8['Durée arrêts']

df_ax8

,StationId,Durée arrêts,Durée autres,Freq arrêts,Freq autres,Duration_20_25 reel
74,75,2444.681944,167.682500,65,106,2612.364444
59,60,1566.194444,132.347500,93,135,1698.541944
64,65,810.854444,176.764444,101,293,987.618889
44,45,745.660833,281.713889,162,257,1027.374722
100,101,735.173611,341.534444,50,141,1076.708056
63,64,709.685000,267.658611,67,160,977.343611
23,24,584.707222,-240.918611,52,203,343.788611
49,50,432.251944,188.302778,259,462,620.554722
113,114,422.650556,330.388333,103,223,753.038889
53,54,328.611944,267.900278,258,197,596.512222


## str ax12

In [194]:
Ep_cumul = cumul_results['wtc_kWG1TotE_accum'].sum()
EL_cumul = cumul_results['EL'].sum()
ELX_cumul = cumul_results['ELX'].sum()
ELNX_cumul = cumul_results['ELNX'].sum()
EL_2006_cumul = cumul_results['EL_2006'].sum()
EL_PowerRed_cumul = cumul_results['EL_PowerRed'].sum()

EL_Misassigned_cumul = cumul_results.query('StationId == 75 | StationId == 65')[
    'EL_Misassigned'].sum()


Epot_cumul = cumul_results['Epot'].sum()

EL_wind_cumul = cumul_results['EL_wind'].sum()
EL_wind_start_cumul = cumul_results['EL_wind_start'].sum()
EL_alarm_start_cumul = cumul_results['EL_alarm_start'].sum()

MAA_cumul = round(100 * (
    Ep_cumul + ELX_cumul) / (
        Ep_cumul + EL_cumul - (EL_wind_cumul + EL_wind_start_cumul + EL_alarm_start_cumul + EL_2006_cumul)), 2)


MAA_cumul_brut = round(100 * (
    Ep_cumul + ELX_cumul) / (
        Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul), 2)


MAA_cumul_brut_mis = round(100 * (
    Ep_cumul + ELX_cumul) / (
        Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul + EL_Misassigned_cumul), 2)


In [195]:
df = cumul_results[['TimeStamp', 'wtc_kWG1TotE_accum',
                    'EL',
                    'ELX',
                    'ELNX',
                    'EL_2006']
                   ].groupby(cumul_results.TimeStamp.dt.month).sum()

Ep_cumul_grp = df['wtc_kWG1TotE_accum']
EL_cumul_grp = df['EL']
ELX_cumul_grp = df['ELX']
ELNX_cumul_grp = df['ELNX']
EL_2006_cumul_grp = df['EL_2006']


In [196]:
(100 * ELX_cumul_grp /
          (Ep_cumul_grp + ELX_cumul_grp + ELNX_cumul_grp + EL_2006_cumul_grp)).mean()


0.36673865282687856

In [197]:
MAA_cumul, MAA_cumul_brut, MAA_cumul_brut_mis


(96.54, 96.61, 96.54)

In [198]:
(
    round(100 * ELX_cumul /
          (Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul), 2),
    round(100 * (ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul) /
          (Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul), 2),
    round(100 * (ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul + EL_Misassigned_cumul) /
          (Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul + EL_Misassigned_cumul), 2)
)


(0.33, 3.39, 3.46)

In [199]:
Ep_period = results['wtc_kWG1TotE_accum'].sum()
EL_period = results['EL'].sum()
ELX_period = results['ELX'].sum()
ELNX_period = results['ELNX'].sum()
EL_2006_period = results['EL_2006'].sum()
EL_PowerRed_period = results['EL_PowerRed'].sum()

EL_Misassigned_period = results.query('StationId == 75 | StationId == 65')[
    'EL_Misassigned'].sum()


Epot_period = results['Epot'].sum()

EL_wind_period = results['EL_wind'].sum()
EL_wind_start_period = results['EL_wind_start'].sum()
EL_alarm_start_period = results['EL_alarm_start'].sum()

MAA_period = round(100 * (
    Ep_period + ELX_period) / (
        Ep_period + EL_period - (EL_wind_period + EL_wind_start_period + EL_alarm_start_period + EL_2006_period)), 2)

MAA_period_brut = round(100 * (
    Ep_period + ELX_period) / (
        Ep_period + ELX_period + ELNX_period + EL_2006_period + EL_PowerRed_period), 2)

MAA_period_brut_mis = round(100 * (
    Ep_period + ELX_period) / (
        Ep_period + ELX_period + ELNX_period + EL_2006_period + EL_PowerRed_period + EL_Misassigned_period), 2)
        


In [200]:
MAA_period, MAA_period_brut, MAA_period_brut_mis


(97.61, 97.94, 97.94)

In [201]:
EL_2006_period


0.0

In [202]:
(
    round(100 * ELX_period /
          (Ep_period + ELX_period + ELNX_period + EL_2006_period + EL_PowerRed_period), 2),
    round(100 * (ELNX_period + EL_2006_period + EL_PowerRed_period) /
          (Ep_period + ELX_period + ELNX_period + EL_2006_period + EL_PowerRed_period), 2),
    round(100 * (ELNX_period + EL_2006_period + EL_PowerRed_period + EL_Misassigned_period) /
          (Ep_period + ELX_period + ELNX_period + EL_2006_period + EL_PowerRed_period + EL_Misassigned_period), 2)
)


(0.19, 2.06, 2.06)

In [204]:
ELNX_brid_cumul = round(
    (ELNX_cumul + EL_2006_cumul + EL_PowerRed_cumul) / 1e6, 2)
ELNX_brid_period = round((ELNX_period + EL_2006_period + EL_PowerRed_period) / 1e6, 2)


In [86]:
# str_ax12 = f"""ELNX imputée à SGRE:  {ELNX_cumul} GWh ({ELNX_period} GWh pour {period_month_name} {period_year})
# Disponibilité en énergie selon FSA : {MAA_cumul_brut}% ({MAA_period_brut} % pour {period_month_name} {period_year})
# Disponibilité en énergie constatée : {MAA_cumul}% ({MAA_period} % pour {period_month_name} {period_year})"""

In [205]:
str_ax12 = f"""Energie imputée à SGRE: {period_year} YTD {ELNX_brid_cumul} GWh & Mensuel {ELNX_brid_period} GWh
Disponibilité en énergie selon FSA: {period_year} YTD {MAA_cumul_brut} % & Mensuel {MAA_period_brut} %"""


In [206]:
str_ax12

'Energie imputée à SGRE: 2021 YTD 21.84 GWh & Mensuel 3.03 GWh\nDisponibilité en énergie selon FSA: 2021 YTD 96.61 % & Mensuel 97.94 %'

## Graph 7 ax18

In [207]:
df_ax18 = (cumul_results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('StationId')
                # .head(20)
                .reset_index())

df_ax18[['ELNX', 'EL_indefini_left']] = df_ax18[['ELNX', 'EL_indefini_left']] / 1e3

df_ax18.rename({'ELNX': 'Energie perdue SGRE'}, inplace=True, axis=1)

df_ax18 = df_ax18.round(2)

In [208]:
df_ax18

,StationId,Energie perdue SGRE,EL_indefini_left
0,1,188.75,-0.00
1,2,114.82,0.45
2,3,144.10,1.78
3,4,78.63,-0.00
4,5,153.56,0.05
...,...,...,...
126,127,168.04,0.20
127,128,149.14,1.56
128,129,36.75,0.09
129,130,67.99,0.42


## ax19

In [209]:
from scipy.stats import binned_statistic
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy import integrate
from scipy.interpolate import interp1d


In [210]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [211]:
CB2 = pd.read_excel('CB2.xlsx')

CB2 = CB2.astype(int).drop_duplicates()

CB2_interp = interp1d(CB2.Wind,
              CB2.Power, kind='linear', 
              fill_value="extrapolate")

In [212]:
# import warnings
# warnings.filterwarnings("error")

def ratio_cp(df, CB2_interp, name):

    df = df.query('RealPeriod == 0')
    
    if len(df) == 0:
        return np.nan
    
    x = df.wtc_AcWindSp_mean.values
    y = df.wtc_ActPower_mean.values.clip(0, 2300)
    
    bin_means, bin_edges, binnumber = (
        binned_statistic(x, y, statistic='mean', bins=25))
    
    bins = running_mean(bin_edges, 2)

    nan_index_means = np.argwhere(
        np.isnan(
            bin_means
        )
    )

    bin_means = np.delete(bin_means, nan_index_means)

    bins = np.delete(bins, nan_index_means)
    
    df_interp = interp1d(bins, bin_means, kind='linear', 
                         fill_value="extrapolate")
    
    nan_index = np.argwhere(
        np.isnan(
            df_interp(bins)
        )
    )

    bins = np.delete(bins, nan_index)
    
    try:
    
        integral_df = quad(df_interp, bins.min(), bins.max(),
                           points=bins,
                           limit=100)
    
        integral_siemens = quad(CB2_interp,
                                bins.min(),bins.max(),
                                points=CB2.Wind, 
                                limit=100)

    except: 
        print('except tur:')
        print(name)
        print(len(df))
        return None
    
    # print('tur')
    # print(name)

    # return 100 * integral_df[0] / integral_siemens[0]

    return 100 * (y.sum() / CB2_interp(x).sum())

    # return (100 * (df.wtc_ActPower_mean / CB2_interp(x))).replace(np.NINF, 100).mean()


    

In [213]:
df_ax19 = (results.groupby('StationId')
                  .apply(lambda df: ratio_cp(df, CB2_interp, df.name))
                  .to_frame('Ratio')
                  .reset_index())

In [214]:
df_ax19.Ratio.mean()

99.23814295468688

In [215]:
df_ax19.iloc[df_ax19.Ratio.idxmin()]

StationId    123.000000
Ratio         89.076071
Name: 122, dtype: float64

In [216]:
df_ax19

,StationId,Ratio
0,1,91.668848
1,2,99.507162
2,3,96.919477
3,4,99.660610
4,5,93.105640
...,...,...
126,127,95.183091
127,128,98.028890
128,129,93.915998
129,130,95.688642


## ratio period et cumul

In [217]:
ratio_period = df_ax19.Ratio.mean()
df_ax1.iat[1, 8] = ratio_period

In [218]:
ratio_cumul = ((last_ratio_cumul*days_to_previous_period + ratio_period*days_in_period)
               /(days_to_previous_period+days_in_period))

df_ax1.iat[0, 8] = ratio_cumul

In [219]:
ratio_cumul, ratio_period


(99.79138577260645, 99.23814295468688)

# Export XLSX writer

In [352]:
writer = pd.ExcelWriter(f'./input/{period}/output_xlsxwriter.xlsx')
# -------------------------------------------------------------------------------------------

workbook = writer.book
dashsheet = workbook.add_worksheet('Dash')

# -------------------------------------------------------------------------------------------

MTBF_MTTR_cat_cumul.to_excel(writer, sheet_name='MTBF & MTTR cumul', index=False)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax1-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax1.to_excel(writer, sheet_name='ax1')

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax3-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax3.to_excel(writer, index=False, sheet_name='ax3')

worksheet = writer.sheets['ax3']


def make_chart_ax3():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax3', 0, 2],
                             'categories': ['ax3', 1, 0, 13, 0],
                             'values': ['ax3', 1, 2, 13, 2], })

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax3', 1, 1, 13, 1],
        'categories': ['ax3', 1, 0, 13, 0],
        'name': ['ax3', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax3.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Cumul annuel par type d\'alarme',
                            'name_font': {'size': 12, 'bold': True}})
    line_chart.set_y2_axis({'name': df_ax3.columns[1]})
    return column_chart


column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B2', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax4-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax4.to_excel(writer, index=False, sheet_name='ax4')
worksheet = writer.sheets['ax4']


def make_chart_ax4():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax4', 0, 2],
                             'categories': ['ax4', 1, 0, 13, 0],
                             'values': ['ax4', 1, 2, 13, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax4', 1, 1, 13, 1],
        'categories': ['ax4', 1, 0, 13, 0],
        'name': ['ax4', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)
    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax4.columns[0]})
    column_chart.set_y_axis({'name': 'MTBF en j'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'MTBF et MTTR par catégorie sur l\'année 2021',
                            'name_font': {'size': 12, 'bold': True}})
    line_chart.set_y2_axis({'name': 'MTTR en h'})
    return column_chart


column_chart = make_chart_ax4()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax4()
# Insert the chart into the worksheet.
dashsheet.insert_chart('J2', column_chart)



# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax5-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax5.to_excel(writer, index=False, sheet_name='ax5')

worksheet = writer.sheets['ax5']


def make_chart_ax5():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax5', 0, 2],
                             'categories': ['ax5', 1, 0, 13, 0],
                             'values': ['ax5', 1, 2, 13, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax5', 1, 1, 13, 1],
        'categories': ['ax5', 1, 0, 13, 0],
        'name': ['ax5', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax5.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Type d\'alarme {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax5.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R2', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax6-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax6.to_excel(writer, index=False, sheet_name='ax6')

worksheet = writer.sheets['ax6']


def make_chart_ax6():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax6', 0, 2],
                             'categories': ['ax6', 1, 0, 20, 0],
                             'values': ['ax6', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax6', 1, 1, 20, 1],
        'categories': ['ax6', 1, 0, 20, 0],
        'name': ['ax6', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax6.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Alarmes {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax6.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
dashsheet.insert_chart('Z2', column_chart)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax7-------------------------------------------------
# -------------------------------------------------------------------------------------------

worksheet = workbook.add_worksheet('ax7')

options = {
    'x_scale': 3,
    'y_scale': 25,
}

worksheet.insert_textbox(0, 0, ax7, options)



# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax8-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax8.to_excel(writer, index=False, sheet_name='ax8')

worksheet = writer.sheets['ax8']


def make_chart_ax8():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax8', 0, 1],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax8', 0, 2],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax8', 1, 3, 20, 3],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax8', 1, 4, 20, 4],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax8.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Arrêts turbines : Cumul Annuel',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax9-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax9.to_excel(writer, index=False, sheet_name='ax9')

worksheet = writer.sheets['ax9']


def make_chart_ax9():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax9', 0, 1],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax9', 0, 2],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax9', 1, 3, 20, 3],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax9', 1, 4, 20, 4],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax9.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Arrêts turbines {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
dashsheet.insert_chart('J17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax10------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax10.to_excel(writer, index=False, sheet_name='ax10')

worksheet = writer.sheets['ax10']


def make_chart_ax10():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax10', 1, col, 20, col],
                                 'categories': ['ax10', 1, 0, 20, 0],
                                 'name': ['ax10', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax10.columns[0]})
    # column_chart.set_y_axis({'name': df_ax10.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': f'Energie perdue selon FSA du {period}',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax11------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax11.to_excel(writer, index=False, sheet_name='ax11')


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax12------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax12.to_excel(writer, sheet_name='ax12')

worksheet = writer.sheets['ax12']

options = {
    'x_scale': 4,
    'y_scale': 1,
}

worksheet.insert_textbox(0, 6, str_ax12, options)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax18------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax18.to_excel(writer, index=False, sheet_name='ax18')

worksheet = writer.sheets['ax18']


def make_chart_ax18():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax18', 1, col, 131, col],
                                 'categories': ['ax18', 1, 0, 131, 0],
                                 'name': ['ax18', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax18.columns[0]})
    # column_chart.set_y_axis({'name': df_ax18.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': 'Energie perdue selon FSA cumulée sur l\'année 2020 en MWh',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B32', column_chart, {'x_scale': 4, 'y_scale': 1})

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax19------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax19.to_excel(writer, index=False, sheet_name='ax19')

worksheet = writer.sheets['ax19']


def make_chart_ax19():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'values': ['ax19', 1, 1, 131, 1],
                             'categories': ['ax19', 1, 0, 131, 0],
                             'name': ['ax19', 0, 1]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax19.columns[0]})
    # column_chart.set_y_axis({'name': df_ax19.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': f'Ratio Courbe Théorique vs Courbe Réelle du {period}',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax19()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax19()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B47', column_chart, {'x_scale': 4, 'y_scale': 1})


writer.save()

In [ ]:
# !jupyter nbconvert performance.ipynb --to markdown